In [21]:
# CS 6316 ML Project: LASSO
# Created on Wed Apr 29 22:23 2020 
# @author: sl4bq

import time
import os
import shutil
import pandas as pd
from datetime import datetime
from dateutil import tz
import warnings
import itertools

import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split



In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import seaborn as sns

In [24]:
# Remove the NaN rows and columns

def removeNaN(curr_feat_1, curr_feat_2):
    curr_feat_1_nan = np.array([])  # numpy.float64
    curr_feat_2_nan = np.array([])  # numpy.float64
    posi = 0
    # *** Check for and remove the Nan rows in from ONLY [curr_feat_2] feature rows
    for posi in range(len(curr_feat_2)):
        if not (math.isnan(curr_feat_2[posi])):# and not(math.isnan(curr_feat_1[posi]))):
            curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
        if (math.isnan(curr_feat_1[posi])):
            # curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            # curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
    return curr_feat_1_nan, curr_feat_2_nan

#### Import .csv dataset into pandas
print('Importing Data Now')

#filename = '/Users/Mandala/PycharmProjects/ML_Project/windowing_results.csv'
filename = 'skin_conductance_new.csv'

# Read data from csv file 
df_feat = pd.read_csv(filename)
df_keys = df_feat.keys()



Importing Data Now


In [25]:
df_feat

,Unnamed: 0,Unnamed: 0.1,GSR #,start_time,end_time,gsr_mean,gsr_median,gsr_variance,gsr_skewness,gsr_kurtosis,Act
0,0,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:27,2019-11-21 09:35:29,1.355378,1.332223,0.000256,0.699293,-0.627312,"(0,0)"
1,1,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:28,2019-11-21 09:35:30,1.339162,1.318903,0.000125,-0.100332,-1.238669,"(0,0)"
2,2,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:29,2019-11-21 09:35:31,1.326166,1.306471,0.000135,0.010484,-1.243451,"(0,0)"
3,3,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:30,2019-11-21 09:35:32,1.313587,1.294483,0.000113,0.140383,-1.149240,"(0,0)"
4,4,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:31,2019-11-21 09:35:33,1.302077,1.286491,0.000085,0.139074,-1.145452,"(0,0)"
...,...,...,...,...,...,...,...,...,...,...,...
10286,10286,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 11:00:35,2019-11-21 11:00:37,1.873922,1.857143,0.000018,1.241958,5.575090,"(2,3)"
10287,10287,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 11:00:36,2019-11-21 11:00:38,1.872439,1.838095,0.000022,0.428993,5.263945,"(2,3)"
10288,10288,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 11:00:37,2019-11-21 11:00:39,1.869794,1.838095,0.000021,0.490440,4.582487,"(2,3)"
10289,10289,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 11:00:38,2019-11-21 11:00:40,1.866879,1.838095,0.000020,0.256191,3.974350,"(2,3)"


In [26]:
print(df_keys)
len(df_keys)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'GSR #', 'start_time', 'end_time',
       'gsr_mean', 'gsr_median', 'gsr_variance', 'gsr_skewness',
       'gsr_kurtosis', 'Act'],
      dtype='object')


11

In [27]:
# Data Preprocessing
df = df_feat.copy()
df = df.dropna()

Xs = df[df_keys[5:10]]
y = df['Act']

print(Xs)
print(y)

       gsr_mean  gsr_median  gsr_variance  gsr_skewness  gsr_kurtosis
0      1.355378    1.332223      0.000256      0.699293     -0.627312
1      1.339162    1.318903      0.000125     -0.100332     -1.238669
2      1.326166    1.306471      0.000135      0.010484     -1.243451
3      1.313587    1.294483      0.000113      0.140383     -1.149240
4      1.302077    1.286491      0.000085      0.139074     -1.145452
...         ...         ...           ...           ...           ...
10286  1.873922    1.857143      0.000018      1.241958      5.575090
10287  1.872439    1.838095      0.000022      0.428993      5.263945
10288  1.869794    1.838095      0.000021      0.490440      4.582487
10289  1.866879    1.838095      0.000020      0.256191      3.974350
10290  1.865581    1.842491      0.000017      0.676028      4.517975

[10291 rows x 5 columns]
0        (0,0)
1        (0,0)
2        (0,0)
3        (0,0)
4        (0,0)
         ...  
10286    (2,3)
10287    (2,3)
10288    (2,3)

In [28]:
# This is the data split

X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=10)

# Training the model
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [29]:
y_pred = model.predict(X_test)
print(y_pred)

['(1,2)' '(0,1)' '(1,2)' ... '(0,1)' '(1,2)' '(1,2)']


In [30]:
len(y_pred)

2059

In [31]:
len(y_test)

2059

In [32]:
accuracy = accuracy_score(y_test, y_pred)*100
print(accuracy)

print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
      .format(X_test.shape[0],(y_test != y_pred).sum(),100*(1-(y_test != y_pred).sum()/X_test.shape[0])))

20.20398251578436
Number of mislabeled points out of a total 2059 points : 1643, performance 20.20%


In [33]:


predicted = model.predict(X_test)

print(np.mean(predicted == y_test))

0.2020398251578436


In [34]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))

[[  0  40   5   0   0   0   3   4   0   2   0   0]
 [  0 132  47   0   0   0  11  24   0   0   4   0]
 [  0 113  78   3   0   0   8  15   0   2   4   0]
 [  0  29  43   5   0   0  24  65   0   5   6   1]
 [  0   3  11   0   0   0  18   5   0   0   7   0]
 [  0  36  55   4   0   0  37  38   0   3  18   0]
 [  0  88   5   1   0   0 105  69   0   3  35   2]
 [  0  50  12   2   0   0  41  54   0   2  26   0]
 [  0  15   1   0   0   0  14   5   0   1   6   2]
 [  0  61   0   2   0   0  31  37   0   4  17   2]
 [  0  86   9   8   0   0  91  45   0   6  37   3]
 [  0  59   2   5   0   0  45  31   0   7  23   1]]


In [35]:
df_video = df.loc[(df.Act == "(0,0)") | (df.Act == "(0,1)") | (df.Act == "(0,2)") | (df.Act == "(0,3)")]
df_noeval = df.loc[(df.Act == "(1,0)") | (df.Act == "(1,1)") | (df.Act == "(1,2)") | (df.Act == "(1,3)")]
df_eval = df.loc[(df.Act == "(2,0)") | (df.Act == "(2,1)") | (df.Act == "(2,2)") | (df.Act == "(2,3)")]

In [36]:
df_video

,Unnamed: 0,Unnamed: 0.1,GSR #,start_time,end_time,gsr_mean,gsr_median,gsr_variance,gsr_skewness,gsr_kurtosis,Act
0,0,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:27,2019-11-21 09:35:29,1.355378,1.332223,0.000256,0.699293,-0.627312,"(0,0)"
1,1,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:28,2019-11-21 09:35:30,1.339162,1.318903,0.000125,-0.100332,-1.238669,"(0,0)"
2,2,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:29,2019-11-21 09:35:31,1.326166,1.306471,0.000135,0.010484,-1.243451,"(0,0)"
3,3,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:30,2019-11-21 09:35:32,1.313587,1.294483,0.000113,0.140383,-1.149240,"(0,0)"
4,4,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:31,2019-11-21 09:35:33,1.302077,1.286491,0.000085,0.139074,-1.145452,"(0,0)"
...,...,...,...,...,...,...,...,...,...,...,...
8617,8617,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 10:07:33,2019-11-21 10:07:35,1.500604,1.482051,0.000018,0.787991,3.596571,"(0,3)"
8618,8618,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 10:07:34,2019-11-21 10:07:36,1.497648,1.482051,0.000015,0.904137,5.416618,"(0,3)"
8619,8619,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 10:07:35,2019-11-21 10:07:37,1.495637,1.480586,0.000012,1.239530,7.910456,"(0,3)"
8620,8620,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 10:07:36,2019-11-21 10:07:38,1.495160,1.480586,0.000010,1.316096,7.396589,"(0,3)"


In [38]:
def NBC(x):
    y = x.copy()
    df = y.dropna()

    Xs = df[df_keys[5:10]]
    y = df['Act']
    
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=10)
    
    model = GaussianNB()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)*100
    print(accuracy)

In [39]:
NBC(df_video)

36.56387665198238


In [40]:
NBC(df_noeval)

38.23129251700681


In [41]:
NBC(df_eval)

42.54658385093168
